In [2]:
import pandas as pd
import numpy as np

# Ham veri dosyalarının yollarını tanımladık.
path_salary_data = "../data/raw/salary_data.csv"
path_salary_by_country = "../data/raw/salary_by_jobtitle_country.csv"


In [3]:
# Kaggle'dan aldığımız iki veri setini okuduk.
df_salary_raw = pd.read_csv(path_salary_data)
df_country_raw = pd.read_csv(path_salary_by_country)

print("salary_data shape:", df_salary_raw.shape)
display(df_salary_raw.head())
display(df_salary_raw.info())

print("\nsalary_by_jobtitle_country shape:", df_country_raw.shape)
display(df_country_raw.head())
display(df_country_raw.info())


salary_data shape: (6704, 6)


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6702 non-null   float64
 1   Gender               6702 non-null   object 
 2   Education Level      6701 non-null   object 
 3   Job Title            6702 non-null   object 
 4   Years of Experience  6701 non-null   float64
 5   Salary               6699 non-null   float64
dtypes: float64(3), object(3)
memory usage: 314.4+ KB


None


salary_by_jobtitle_country shape: (6684, 9)


,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000.0,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000.0,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000.0,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000.0,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000.0,USA,Asian,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6684 entries, 0 to 6683
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6684 non-null   float64
 1   Gender               6684 non-null   object 
 2   Education Level      6684 non-null   int64  
 3   Job Title            6684 non-null   object 
 4   Years of Experience  6684 non-null   float64
 5   Salary               6684 non-null   float64
 6   Country              6684 non-null   object 
 7   Race                 6684 non-null   object 
 8   Senior               6684 non-null   int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 470.1+ KB


None

In [4]:
# Kolon isimlerini ortak bir standarda çektik ki iki veri setini aynı şema altında rahatça kullanabilelim.
def standardize_columns(df):
    rename_map = {
        'Age': 'age',
        'Gender': 'gender',
        'Education Level': 'education_level',
        'Job Title': 'job_title',
        'Years of Experience': 'years_experience',
        'Salary': 'salary',
        'Country': 'country',
        'Race': 'race',
        'Senior': 'senior'
    }
    return df.rename(columns=rename_map)

df_salary = standardize_columns(df_salary_raw)
df_country = standardize_columns(df_country_raw)

df_salary.head(), df_country.head()


(    age  gender education_level          job_title  years_experience    salary
 0  32.0    Male      Bachelor's  Software Engineer               5.0   90000.0
 1  28.0  Female        Master's       Data Analyst               3.0   65000.0
 2  45.0    Male             PhD     Senior Manager              15.0  150000.0
 3  36.0  Female      Bachelor's    Sales Associate               7.0   60000.0
 4  52.0    Male        Master's           Director              20.0  200000.0,
     age  gender  education_level          job_title  years_experience  \
 0  32.0    Male                1  Software Engineer               5.0   
 1  28.0  Female                2       Data Analyst               3.0   
 2  45.0    Male                3            Manager              15.0   
 3  36.0  Female                1    Sales Associate               7.0   
 4  52.0    Male                2           Director              20.0   
 
      salary country      race  senior  
 0   90000.0      UK     White  

In [5]:
# Seniority bilgisini her iki veri seti için de ortak bir kolon altında topladık (seniority_level).
# Varsayılan olarak herkes "junior", gerçekten senior olduğu belli olanları "senior" yaptık.
def extract_seniority_level(df, has_senior_column):
    df = df.copy()
    df['seniority_level'] = 'junior'

    # Senior kolonu olan veri seti için (by_country) bu kolonu kullandık.
    if has_senior_column and 'senior' in df.columns:
        s = df['senior'].astype(str).str.lower()
        mask_senior_col = (s == '1') | s.str.contains('1') | s.str.contains('true') | s.str.contains('yes')
        df.loc[mask_senior_col, 'seniority_level'] = 'senior'

    # Job title içinden de senior/junior bilgisini çıkardık.
    if 'job_title' in df.columns:
        jt_lower = df['job_title'].astype(str).str.lower()

        # Title içinde "senior" geçenleri senior olarak işaretledik.
        mask_senior_title = jt_lower.str.contains('senior ')
        df.loc[mask_senior_title, 'seniority_level'] = 'senior'

        # Title içinde "junior" geçenleri junior olarak bıraktık.
        mask_junior_title = jt_lower.str.contains('junior ')
        df.loc[~mask_senior_title & mask_junior_title, 'seniority_level'] = 'junior'

        # Maaş dağılımlarını analiz ettik ve ne senior ne junior yazmayanların "junior" olduğunu tespit ettik.  (neutral = junior kararı).

    return df

df_salary = extract_seniority_level(df_salary, has_senior_column=False)
df_country = extract_seniority_level(df_country, has_senior_column=True)

df_salary[['job_title', 'seniority_level']].head(), df_country[['job_title', 'seniority_level']].head()


(           job_title seniority_level
 0  Software Engineer          junior
 1       Data Analyst          junior
 2     Senior Manager          senior
 3    Sales Associate          junior
 4           Director          junior,
            job_title seniority_level
 0  Software Engineer          junior
 1       Data Analyst          junior
 2            Manager          senior
 3    Sales Associate          junior
 4           Director          junior)

In [6]:
# Kıdem bilgisini ayrı bir kolona aldığımız için job_title içindeki senior/junior ifadelerini temizledik.
# Böylece job_title sadece pozisyonu temsil ediyor.
def normalize_job_title(df):
    df = df.copy()
    if 'job_title' not in df.columns:
        return df

    jt = df['job_title'].astype(str).str.lower()

    # Seniority ile ilgili kelimeleri title'dan çıkardık.
    for token in ['senior', 'junior', 'sr.', 'sr', 'jr.', 'jr']:
        jt = jt.str.replace(token, '', regex=False)

    # Fazla boşlukları tek boşluğa indirdik ve sağdan soldan kırptık.
    jt = jt.str.replace(r'\s+', ' ', regex=True).str.strip()

    df['job_title'] = jt
    return df

df_salary = normalize_job_title(df_salary)
df_country = normalize_job_title(df_country)

df_salary['job_title'].value_counts().head(), df_country['job_title'].value_counts().head()


(job_title
 software engineer            813
 data scientist               515
 data analyst                 391
 software engineer manager    376
 product manager              323
 Name: count, dtype: int64,
 job_title
 software engineer            809
 data scientist               515
 data analyst                 391
 software engineer manager    376
 product manager              323
 Name: count, dtype: int64)

In [7]:
# Eğitim seviyelerini de her iki veri seti için ortak 4 kategoriye çevirdik:
# high_school, bachelor, master, phd
def normalize_education_level(df):
    df = df.copy()

    def map_edu(val):
        if pd.isna(val):
            return 'unknown'

        # Sayısal kodlar için (0,1,2,3) gelen kayıtları map'ledik.
        if isinstance(val, (int, float)) and not isinstance(val, bool):
            if val == 0:
                return 'high_school'
            elif val == 1:
                return 'bachelor'
            elif val == 2:
                return 'master'
            elif val == 3:
                return 'phd'
            else:
                return 'unknown'

        # Metin ifadeleri küçük harfe çevirip eşleştirdik.
        s = str(val).strip().lower()
        if 'high school' in s:
            return 'high_school'
        if 'bachelor' in s:
            return 'bachelor'
        if 'master' in s:
            return 'master'
        if 'phd' in s:
            return 'phd'
        return 'unknown'

    if 'education_level' in df.columns:
        df['education_level'] = df['education_level'].apply(map_edu)

    return df

df_salary = normalize_education_level(df_salary)
df_country = normalize_education_level(df_country)

df_salary['education_level'].value_counts(), df_country['education_level'].value_counts()


(education_level
 bachelor       3023
 master         1861
 phd            1369
 high_school     448
 unknown           3
 Name: count, dtype: int64,
 education_level
 bachelor       3021
 master         1858
 phd            1369
 high_school     436
 Name: count, dtype: int64)

In [8]:
# Modelde kullanacağımız çekirdek kolonları tanımladık.
core_cols = ['age', 'job_title', 'education_level', 'years_experience', 'salary', 'seniority_level']

def select_core_columns(df):
    """
    Veri setini modelde kullanacağım çekirdek kolonlara indirdil.
    Gender, country, race, senior gibi kolonları bu aşamadan sonra kullanmıyoruz.
    """
    df = df.copy()
    for col in core_cols:
        if col not in df.columns:
            df[col] = np.nan
    return df[core_cols].copy()

df_salary_core = select_core_columns(df_salary)
df_country_core = select_core_columns(df_country)

df_salary_core.head(), df_country_core.head()


(    age          job_title education_level  years_experience    salary  \
 0  32.0  software engineer        bachelor               5.0   90000.0   
 1  28.0       data analyst          master               3.0   65000.0   
 2  45.0            manager             phd              15.0  150000.0   
 3  36.0    sales associate        bachelor               7.0   60000.0   
 4  52.0           director          master              20.0  200000.0   
 
   seniority_level  
 0          junior  
 1          junior  
 2          senior  
 3          junior  
 4          junior  ,
     age          job_title education_level  years_experience    salary  \
 0  32.0  software engineer        bachelor               5.0   90000.0   
 1  28.0       data analyst          master               3.0   65000.0   
 2  45.0            manager             phd              15.0  150000.0   
 3  36.0    sales associate        bachelor               7.0   60000.0   
 4  52.0           director          master   

In [9]:
# Yaş, deneyim yılı ve maaşı sayısal tipe çevirdik. Dönüşemeyenleri NaN yaptık.
def convert_numeric(df):
    df = df.copy()
    for col in ['age', 'years_experience', 'salary']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

df_salary_core = convert_numeric(df_salary_core)
df_country_core = convert_numeric(df_country_core)

df_salary_core.info(), df_country_core.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               6702 non-null   float64
 1   job_title         6704 non-null   object 
 2   education_level   6704 non-null   object 
 3   years_experience  6701 non-null   float64
 4   salary            6699 non-null   float64
 5   seniority_level   6704 non-null   object 
dtypes: float64(3), object(3)
memory usage: 314.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6684 entries, 0 to 6683
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               6684 non-null   float64
 1   job_title         6684 non-null   object 
 2   education_level   6684 non-null   object 
 3   years_experience  6684 non-null   float64
 4   salary            6684 non-null   float64
 5   seniority_level   6684 n

(None, None)

In [10]:
# Yaş, deneyim ve maaş için temel mantık filtrelerini uyguladık.
def basic_sanity_filters(df):
    df = df.copy()

    # Yaş aralığını 18-70 bandına çektik.
    df = df[(df['age'].isna()) | ((df['age'] >= 18) & (df['age'] <= 70))]

    # Deneyim yılı için 0-50 aralığını koruduk.
    df = df[(df['years_experience'].isna()) |
            ((df['years_experience'] >= 0) & (df['years_experience'] <= 50))]

    # Maaşı pozitif ve makul aralıkta tuttuk.
    df = df[df['salary'] > 0]

    # En üst yüzde 1'lik dilimi uç değer kabul edip temizledik.
    upper_cap = df['salary'].quantile(0.99)
    df = df[df['salary'] <= upper_cap]

    return df

df_salary_core = basic_sanity_filters(df_salary_core)
df_country_core = basic_sanity_filters(df_country_core)

df_salary_core.describe(), df_country_core.describe()


(               age  years_experience         salary
 count  6643.000000       6643.000000    6643.000000
 mean     33.514677          7.990291  114413.719253
 std       7.545998          5.969152   52050.063852
 min      21.000000          0.000000     350.000000
 25%      28.000000          3.000000   70000.000000
 50%      32.000000          7.000000  115000.000000
 75%      38.000000         12.000000  160000.000000
 max      62.000000         34.000000  210000.000000,
                age  years_experience         salary
 count  6628.000000       6628.000000    6628.000000
 mean     33.502867          7.972616  114391.695685
 std       7.525671          5.937356   52069.112774
 min      21.000000          0.000000     350.000000
 25%      28.000000          3.000000   70000.000000
 50%      32.000000          7.000000  115000.000000
 75%      38.000000         12.000000  160000.000000
 max      62.000000         34.000000  210000.000000)

In [11]:
# Eksik değerler için belirlediğim stratejiyi uyguladık.
def handle_missing_values(df):
    df = df.copy()

    # Maaşı eksik kayıtları tamamen çıkardık.
    df = df[~df['salary'].isna()]

    # Kategorik kolonlardaki eksikleri 'unknown' ile doldurduk.
    for col in ['job_title', 'education_level', 'seniority_level']:
        df[col] = df[col].fillna('unknown')

    # Sayısal kolonlardaki eksikleri median ile tamamladık.
    for col in ['age', 'years_experience']:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

    return df

df_salary_core = handle_missing_values(df_salary_core)
df_country_core = handle_missing_values(df_country_core)

df_salary_core.isna().sum(), df_country_core.isna().sum()


(age                 0
 job_title           0
 education_level     0
 years_experience    0
 salary              0
 seniority_level     0
 dtype: int64,
 age                 0
 job_title           0
 education_level     0
 years_experience    0
 salary              0
 seniority_level     0
 dtype: int64)

In [12]:
# İki veri setini çekirdek şema altında tek bir DataFrame'de birleştirdik.
combined = pd.concat([df_salary_core, df_country_core], ignore_index=True)
print("Birleştirme öncesi toplam gözlem sayısı:", combined.shape[0])

# Aynı kaydın birden fazla kez yer almasını engellemek için duplicate kontrolü yaptık.
dup_subset = ['age', 'job_title', 'education_level', 'years_experience', 'salary', 'seniority_level']
num_dups = combined.duplicated(subset=dup_subset).sum()
print("Potansiyel duplicate kayıt sayısı:", num_dups)

combined = combined.drop_duplicates(subset=dup_subset, keep='first')
print("Duplicate temizliği sonrası toplam gözlem sayısı:", combined.shape[0])

combined.head()


Birleştirme öncesi toplam gözlem sayısı: 13271
Potansiyel duplicate kayıt sayısı: 11582
Duplicate temizliği sonrası toplam gözlem sayısı: 1689


,age,job_title,education_level,years_experience,salary,seniority_level
0,32.0,software engineer,bachelor,5.0,90000.0,junior
1,28.0,data analyst,master,3.0,65000.0,junior
2,45.0,manager,phd,15.0,150000.0,senior
3,36.0,sales associate,bachelor,7.0,60000.0,junior
4,52.0,director,master,20.0,200000.0,junior


In [13]:
# Tüm temizlik ve birleştirme adımlarından sonra nihai veri setini diske kaydettik.
output_path = "../data/processed/main_salary_dataset.csv"
combined.to_csv(output_path, index=False)

print("Birleştirilmiş ve temizlenmiş veri setini kaydettim:")
print(output_path)

print("\nSeniority dağılımı:")
print(combined['seniority_level'].value_counts())

print("\nJob title örnekleri:")
print(combined['job_title'].value_counts().head(10))

print("\nEducation level dağılımı:")
print(combined['education_level'].value_counts())


Birleştirilmiş ve temizlenmiş veri setini kaydettim:
../data/processed/main_salary_dataset.csv

Seniority dağılımı:
seniority_level
junior    1327
senior     362
Name: count, dtype: int64

Job title örnekleri:
job_title
software engineer            172
software engineer manager    123
full stack engineer          119
project engineer              95
data scientist                84
back end developer            79
front end developer           68
marketing manager             65
product manager               54
data analyst                  50
Name: count, dtype: int64

Education level dağılımı:
education_level
bachelor       722
master         547
phd            317
high_school    102
unknown          1
Name: count, dtype: int64


In [14]:
# === JOB TITLE ADVANCED NORMALIZATION ===

# 1) Yazım hatası düzeltme
typo_map = {
    "juniour hr coordinator": "junior hr coordinator",
    "juniour hr generalist": "junior hr generalist",
    "social media man": "social media manager",
}

# 2) Benzer rolleri gruplayarak tek kategoriye düşürme
merge_map = {
    "customer service rep": "customer service representative",
    "customer success rep": "customer success manager",
    "developer": "software developer",
    "front end developer": "frontend developer",
    "back end developer": "backend developer",
    "full stack engineer": "fullstack engineer",
}

# 3) Daha tutarlı isimlendirme için ek standardizasyon
standardize_map = {
    "it support": "it support specialist",
    "it project manager": "project manager",
    "research director": "director of research",
    "director": "general director",
    "scientist": "research scientist"
}

# Tüm map’leri birleştir
jobtitle_map = {**typo_map, **merge_map, **standardize_map}

# Uygula
combined["job_title"] = combined["job_title"].replace(jobtitle_map)


In [15]:
# Tekrar kaydetme (temizlik sonrası)
output_path = "../data/processed/main_salary_dataset.csv"
combined.to_csv(output_path, index=False)

print("Revize edilmiş nihai veri seti kaydedildi:")
print(output_path)

print("\nGüncel seniority dağılımı:")
print(combined['seniority_level'].value_counts())

print("\nGüncel job title örnekleri:")
print(combined['job_title'].value_counts().head(20))

print("\nGüncel education level dağılımı:")
print(combined['education_level'].value_counts())


Revize edilmiş nihai veri seti kaydedildi:
../data/processed/main_salary_dataset.csv

Güncel seniority dağılımı:
seniority_level
junior    1327
senior     362
Name: count, dtype: int64

Güncel job title örnekleri:
job_title
software engineer            172
software engineer manager    123
fullstack engineer           119
project engineer              95
data scientist                84
backend developer             79
frontend developer            68
marketing manager             65
product manager               54
data analyst                  50
web developer                 42
sales associate               39
hr generalist                 38
sales representative          36
marketing coordinator         34
software developer            33
operations manager            30
financial manager             28
director of hr                27
human resources manager       27
Name: count, dtype: int64

Güncel education level dağılımı:
education_level
bachelor       722
master         547
ph